# Ejemplo: Algoritmos genéticos (AGs)

## Función de Beale


La función de Beale es una función de dos variables y está definida como sigue:

\begin{equation}
  \min f(x_1, x_2) = (1.5 - x_1 + x_1 x_2)^2 + (2.25 - x_1 + x_1 x_2^2)^2 + (2.625 - x_1 + x_1 x_2^3)^2
\end{equation}

donde $-4.5 \leq x_1, x_2 \leq 4.5$. El mínimo global está en $x^* = (3, 0.5)$ y $f(x^*)=0$.

## Principales componentes de AGs 

Librerías de **Python** que vamos a utilizar.

In [1]:
from random import randint, random, shuffle
from copy import copy, deepcopy
from math import ceil, log, e, exp, sqrt, pi, cos
import numpy as np

Datos de entrada.

In [2]:
n = 2 #Número de variables de decisión
precision = 3 #número de dígitos de precisión
bounds = [-4.5, 4.5] #límites inferior y superior

### Representación de la información genética

La fórmula vista para calcular el número de bits que se necesitan por variable es:

num_bits $ =  \lceil \log_2 ((l_{sup} - l_{inf})*10^{p})\rceil$

donde $p$ es la precisión deseada y $l_{inf}$ y $l_{sup}$ son los límites inferior  y superior de la variable.

In [3]:
def getSizeVariable(bounds, p):
    return ceil(log((bounds[1] - bounds[0])*(10**p), 2))

In [4]:
num_bits = getSizeVariable(bounds, precision)
print(num_bits)

14


Dado que nuestro problema tiene dos variables y ambas tienen el mismo límite superior e inferior, el tamaño de la cadena binaria que representa el cromosoma de un individuo es de longitud $28$. Por ejemplo:

In [5]:
#Si n = 2
i = '0111010000100011010101011001'
print(i, len(i))

0111010000100011010101011001 28


Para poder calcular la aptitud de cada individuo es necesario obtener el valor real de cada una de sus variables.

In [6]:
def getRealValues(chromosome, num_bits, num_variables, bounds):
    point = []
    for i in range(num_variables):
        b = chromosome[i*num_bits:(i+1)*num_bits]

        #binario a entero
        d = 0
        e = num_bits-1
        for j in b:
            d += (int(j) * 2**e)
            e -= 1

        #entero a decimal
        r = bounds[0] + (((bounds[1]-bounds[0])*d)/(2**num_bits-1))
        point.append(r)

    return np.array(point)

In [7]:
i_real = getRealValues(i, 14, 2, bounds)
i_real

array([-0.41723128,  3.00247207])

### Función objetivo / Aptitud

Función de Beale

In [8]:
def f(x):
    x1, x2 = x[0], x[1]
    term1 = (1.5 - x1 + x1*x2)**2
    term2 = (2.25 - x1 + x1*(x2**2))**2
    term3 = (2.625 - x1 + x1*(x2**3))**2

    return term1 + term2 + term3

Dado que estamos resolviendo un problema de minimización, asignamos la aptitud de cada individuo $i$ de la siguiente forma:

$f_a(i)=\frac{1}{f(x_1^{(i)}, x_2^{(i)})+1}$

In [9]:
def getFitness(f_x):
    return 1/(f_x+1)

In [10]:
fx = f(i_real)
print (f"x = {i_real}\nf(x) = {fx}\nAptitud(x) = {getFitness(fx)}")

x = [-0.41723128  3.00247207]
f(x) = 69.71559241216701
Aptitud(x) = 0.014141152833331061


### Representación de un individuo

Para almacenar la información de los individuos, vamos a crear una clase con los siguientes atributos:

 1. **x**: 
Cadena binaria con el cromosoma del individuo.
 2. **x_real**:
Valores reales de cada variable.
 3. **f**:
Valor en la función objetivo.
 4. **fitness**:
Aptitud del individuo.

Y tendrá la función **\_\_str\_\_**.

In [11]:
class Individual:
    def __init__(self, chromosome):
        self.x = chromosome
        self.x_real = None
        self.f = None
        self.fitness = None
        
    def __str__(self):
        #Imprime información del individuo. 
        #Este método se manda a llamar a través print
        return "Cadena binaria: {}\nComponentes reales: {}\nf(x): {}\nAptitud: {}".format(self.x, self.x_real, self.f, self.fitness)

### Población inicial

Para cada posición de la cadena binaria de un individuo, se utiliza una distribución uniforme para generar un número aleatorio en el intervalo $(0, 1)$. Si el valor resultante es menor a 0.5, toma el valor de cero. De lo contrario, toma el valor de 1.

In [12]:
def getInitialPopulation(mu, num_bits, n):
    population = []
    for _ in range(mu):
        chromosome = ""
        for i in range(n*num_bits):
            if random()<.5:
                chromosome+="0"
            else:
                chromosome+="1"
            
        population.append(Individual(chromosome))
    return population

In [13]:
mu = 100
population = getInitialPopulation(mu, num_bits, n)

Evaluamos cada uno de los individuos:

In [14]:
for p in population:
    p.x_real = getRealValues(p.x, num_bits, n, bounds)
    p.f = f(p.x_real)
    p.fitness = getFitness(p.f)

In [15]:
print(population[0])

Cadena binaria: 0011001010100001101010000101
Componentes reales: [-2.72010621 -0.77046329]
f(x): 94.56586667106828
Aptitud: 0.010463987141370645


### Selección de padres

Universal estocástica

In [16]:
def getPopulationMean(population, mu):
    mean = 0
    
    for p in range(mu):
        mean += population[p].fitness
    
    return mean/mu   

In [17]:
#Regresa un arreglo que contiene los índices de los individuos
#que serán padres
def universalStochastic(population, mu):
    r = random() 
    mean = getPopulationMean(population, mu)
    cont = 0
    parents = []
    for i in range(mu):
        cont += population[i].fitness/mean        
        while cont > r and len(parents) < mu: 
            parents.append(i) 
            r += 1
    
    return parents 

In [18]:
parents = universalStochastic(population, mu)
print(population[parents[0]])
print(population[parents[10]])

Cadena binaria: 0011001010100001101010000101
Componentes reales: [-2.72010621 -0.77046329]
f(x): 94.56586667106828
Aptitud: 0.010463987141370645
Cadena binaria: 1110000101000110001111010010
Componentes reales: [3.41997803 0.53753891]
f(x): 0.1092864247827332
Aptitud: 0.9014804271095824


### Recombinación

Cruza de dos puntos: 
1. Generamos dos puntos de cruza aleatorios con distribución uniforme.
2. A partir de la posición $0$ y hasta el primer punto de cruza, el primer hijo toma la información del primer padre y el segundo hijo toma la información del segundo padre.
3. A partir del primer punto de cruza y hasta el segundo punto de cruza, el primer hijo toma la información del segundo padre y el segundo hijo toma la información del primer padre.
4. A partir del segundo punto de cruza y hasta el final de la cadena, el primer hijo toma la información del primer padre y el segundo hijo toma la información del segundo padre.

In [19]:
def twoPointsCrossover(p1, p2):
    index1 = randint(1, len(p2.x)-1)
    index2 = index1
    while index2 == index1:
        index2 = randint(1,len(p2.x)-1)

    if index2 < index1:
        index2, index1 = index1, index2

    offspring_1 = p1.x[:index1]+p2.x[index1:index2]+p1.x[index2:]
    offspring_2 = p2.x[:index1]+p1.x[index1:index2]+p2.x[index2:]

    return Individual(offspring_1), Individual(offspring_2)

In [20]:
o1, o2 = twoPointsCrossover(population[parents[0]], population[parents[10]])
o1.x_real = getRealValues(o1.x, num_bits, n, bounds)
o2.x_real = getRealValues(o2.x, num_bits, n, bounds)
o1.f = f(o1.x_real)
o2.f = f(o2.x_real)
o1.fitness = getFitness(o1.f)
o2.fitness = getFitness(o2.f)
print(o1)
print(o2)

Cadena binaria: 0011001010100001001010000101
Componentes reales: [-2.72010621 -1.89553195]
f(x): 680.8091084998717
Aptitud: 0.0014666861846422343
Cadena binaria: 1110000101000110101111010010
Componentes reales: [3.41997803 1.66260758]
f(x): 305.4954578474802
Aptitud: 0.0032626910918126064


### Mutación

Por cada elemento en la cadena binaria, generamos un número aleatorio en el intervalo $(0,1)$, utilizando una distribución uniforme. Si el número generado es menor que la probabilidad de mutación, se mutará el elemento: si es 1 se hace 0 y si es 0 se hace 1.

In [21]:
def mutation(individual, pm):
    for i in range(len(individual.x)):
        if random() < pm:
            if individual.x[i] == "1":
                individual.x = individual.x[:i] + "0" + individual.x[i+1:]
            else:
                individual.x = individual.x[:i] + "1" + individual.x[i+1:]


In [22]:
print(f"Antes de mutar\n{o1.x}")
mutation(o1, 0.1)
print(f"\nDespués de mutar\n{o1.x}")

Antes de mutar
0011001010100001001010000101

Después de mutar
0011001010100011101010000101


### Selección de sobrevivientes

Selección basada en edad aplicando elitismo: Si el peor de los hijos es peor que la mejor solución de la población actual, se sustituye el peor de los hijos por la mejor solución actual. La siguiente generación estará conformada por los hijos generados.

In [23]:
def getBestIndividual(population):
    best = 0
    for i in range(1, len(population)):
        if population[i].fitness > population[best].fitness:
            best = i
    return best

def getWorstIndividual(population):
    worst = 0
    for i in range(1, len(population)):
        if population[i].fitness < population[worst].fitness:
            worst = i
    return worst

In [24]:
def getSurvivors(parents, offspring):
    best_ind = getBestIndividual(parents)
    worst_ind = getWorstIndividual(offspring)
    
    if parents[best_ind].fitness > offspring[worst_ind].fitness:
        offspring.pop(worst_ind)
        offspring.append(copy(parents[best_ind]))
    
    return offspring

## Algoritmo Genético

A continuación se muestra el algoritmo genético completo.

In [40]:
def GeneticAlgorithm(G, mu, pc, pm, n, precision, bounds):
    num_bits = getSizeVariable(bounds, precision)

    population = getInitialPopulation(mu, num_bits, n)
    #Fitness
    for p in population:
        p.x_real = getRealValues(p.x, num_bits, n, bounds)
        p.f = f(p.x_real)
        p.fitness = getFitness(p.f)

    for _ in range(G):
        parents = universalStochastic(population, mu)
        shuffle(parents)
        offspring = []
        for i in range(0,mu,2):
            if random() < pc:
                o1, o2 = twoPointsCrossover(population[parents[i]], population[parents[i+1]])  
            else:
                o1, o2 = deepcopy(population[parents[i]]), deepcopy(population[parents[i+1]])
                
            mutation(o1, pm)
            o1.x_real = getRealValues(o1.x, num_bits, n, bounds)
            o1.f = f(o1.x_real)
            o1.fitness = getFitness(o1.f)

            mutation(o2, pm)
            o2.x_real = getRealValues(o2.x, num_bits, n, bounds)
            o2.f = f(o2.x_real)
            o2.fitness = getFitness(o2.f)
                
            offspring.extend([o1, o2])
        
        population = getSurvivors(population, offspring)


    best = getBestIndividual(population)
    return population[best]

In [41]:
G = 100 #Número de generaciones
mu = 100 #tamaño de la población
pc = 0.9 #probabilidad de cruza
pm = 0.0001 #probabilidad de mutación
n = 2 #número de variables de decision
precision = 3 #número de dígitos de precisión
bounds = [-4.5, 4.5] #límites inferior y superior

In [42]:
print(GeneticAlgorithm(G, mu, pc, pm, n, precision, bounds))

Cadena binaria: 1101111000001010010000000110
Componentes reales: [3.30626259 0.56610511]
f(x): 0.010923175615842709
Aptitud: 0.9891948509250582


Realizamos $m$ ejecuciones de nuestro algoritmo y obtenemos la corrida que encontró la mejor solución, la corrida que encontró la peor solución, la corrida que se encuentra en la mediana de las $m$ ejecuciones, la media y la desviación estándar de las $m$ corridas con respecto al valor de la función objetivo.

In [31]:
def makeMRuns(m, G, mu, pc, pm, n, precision, bounds):
    sol =[]
    for m in range(m):
        sol.append(GeneticAlgorithm(G, n, pc, pm, n, precision, bounds))


    sol.sort(key=lambda individuo: individuo.f)
    print("*************** Mejor solución ***************")
    print(sol[0])

    print("\n*************** Peor solución ****************")
    print(sol[-1])

    print("\n****************** Mediana *******************")
    print(sol[m//2])

    f_sol = [x.f for x in sol]
    print("\nMedia: ", np.mean(f_sol))
    print("Desviación estándar: ", np.std(f_sol))

In [34]:
m = 21
makeMRuns(m, G, mu, pc, pm, n, precision, bounds)

*************** Mejor solución ***************
Cadena binaria: 1101110110010010010000100001
Componentes reales: [3.28978209 0.58093756]
f(x): 0.020091304576727458
Aptitud: 0.9803044056090018

*************** Peor solución ****************
Cadena binaria: 1100001010101000000110001011
Componentes reales: [ 2.3438015  -4.28300678]
f(x): 35765.95829927722
Aptitud: 2.7958765507331612e-05

****************** Mediana *******************
Cadena binaria: 0101011111110010001011111110
Componentes reales: [-1.40825856  0.42107673]
f(x): 32.408936103276446
Aptitud: 0.029932111483847255

Media:  3412.875542684207
Desviación estándar:  8560.413403038703


Es importante mencionar que este es un diseño simple de un AG. Si se revisa el número de copias que se están realizando de los mejores individuos, se observará que es un valor muy alto y esto puede provocar convergencia a soluciones no óptimas.